In [1]:
import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc_old_segments")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc_old_segments/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
Using custom data configuration corpus-48c43d9b917d9e96
Reusing dataset text (/Users/Gianni/.cache/huggingface/datasets/text/corpus-48c43d9b917d9e96/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|██████████| 3/3 [00:00<00:00, 222.79it/s]
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at models/en_bnc_old_segments/paradigms/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further traini

In [9]:
import tensorflow as tf
import numpy as np

In [3]:
sent = semiotic.corpus.train["text"][10000]
sent

'he still had relatives in newcastle but hadbeen unableto visit them because he believed he wouldbe arrested'

In [119]:
sent_len = len(sent.split())
sent_mask = [" ".join([t if i!=n else "[MASK]" for i,t in enumerate(sent.split())]) for n in range(sent_len)]

In [120]:
input = semiotic.paradigmatic.bert_tokenizer(sent_mask)
outputs = semiotic.paradigmatic.model(input["input_ids"])


In [121]:
# Gather all the masked logits in a batch, by taking the row diagonal of the 3D tensor

parad_logits = tf.gather_nd(outputs.logits, indices=[[i,i] for i in range(sent_len)])

In [122]:
logits_positif = tf.nn.relu(parad_logits)

In [123]:
probs, norms = tf.linalg.normalize(logits_positif, ord=1, axis=1)

In [134]:
# Maybe its cheaper to compute top k for k = non_zero, which would differ from row to row, and hence not yeld a tensor. To be tested.

non_zeroes = tf.math.count_nonzero(parad_data.values,axis=1)
max_non_zeroes = max(non_zeroes).numpy()

parad_data = tf.math.top_k(probs, k = max_non_zeroes, sorted=True)

In [141]:
parad_tokens = [semiotic.syntagmatic.tokenizer.decode(parad.numpy()[:parad_len]).split() for parad, parad_len in zip(parad_data.indices, non_zeroes)]
parad_tokens[0]

['and',
 'had',
 'he',
 'us',
 'it',
 'himself',
 'them',
 'ed',
 'if',
 'sheknew',
 'her',
 'all',
 'listen',
 'delight',
 'why',
 'there',
 'she',
 'herself',
 'murder',
 'a',
 'then',
 'but',
 'what',
 'again',
 'thought',
 'er',
 'lik',
 'uncle',
 'le',
 'ist',
 'ell',
 'talk',
 'urg',
 's',
 'mark',
 'shed',
 'y',
 'acknowledg',
 'watch',
 'edhim',
 'on',
 'rank',
 'back',
 'as',
 'whether',
 'or',
 'think',
 'who',
 'out',
 'ing',
 'youknow',
 'happened',
 'thathe',
 'not',
 'wasnot',
 'understand',
 'em',
 'ard',
 'rush',
 'that',
 'contract',
 'act',
 'we',
 'man',
 'aman',
 'contact',
 'i',
 'say',
 'tell',
 'was',
 'his',
 'ok',
 'ad',
 'hed',
 'touch',
 'ask',
 'knew',
 'dress',
 'ever',
 'lock',
 'id',
 'persuad',
 'never',
 'forhim',
 'how',
 'now',
 'ow',
 'well',
 'work',
 'so',
 'hehad',
 'attack',
 'love',
 'in',
 'whom',
 'ate',
 'ir',
 'miss',
 'either',
 'wish',
 'up',
 'suit',
 'add',
 'bow',
 'question',
 'believe',
 'someone',
 'an',
 'en',
 'anger',
 'aunt',
 'i

In [140]:
slg.util.plot_scatter_line(list(range(max_non_zeroes)),parad_data.values[0],sent.split()[0], add_trace=[(list(range(max_non_zeroes)), v, t) for v, t in zip(parad_data.values[1:],sent.split()[1:])])